# Results for paper: TITLE

In [ ]:
from openfermion import get_sparse_operator
from scipy.sparse.linalg import eigsh
from pathlib import Path
from nbed.utils import save_ordered_xyz_file, pubchem_mol_geometry

def ground(hamiltonian):
    sparse_ham = get_sparse_operator(hamiltonian)
    eigenvalue, _ = eigsh(sparse_ham, k=1, which='SA')
    return eigenvalue[0]

def find_and_save(mol_name):
    if Path(f'./molecular_structures/{mol_name}.xyz').exists():
        print(f'xyz file for {mol_name} already exists.\n')
        return

    geo_dict = pubchem_mol_geometry(mol_name)
    print('Molecule geometry:')
    print(geo_dict)
    save_ordered_xyz_file(mol_name, geo_dict, [0])
    print(f'xyz file for {mol_name} created.\n')

def print_results(result, active):
    
    print(f"Qubits\t\t{result[active]['mu']['n_qubits']}/{result[active]['huz']['n_qubits']}/{result['full']['n_qubits']}")
    print(f"Terms\t\t{result[active]['mu']['terms']}/{result[active]['huz']['terms']}/{result['full']['terms']}")
    print(f'DFT:\t\t{result["e_dft"]}')
    #print(f"CCSD Emb:\t{result['mu']['e_ccsd']}/{result['huz']['e_ccsd']}")
    print(f"Mu Emb:\t\t{result[active]['mu']['ground']}")
    print(f"Huz Emb:\t{result[active]['huz']['ground']}")
    print(f"Diagonalized:\t{result['full']['ground']}")
    print("\n")

def save_data(result, mol_name, localisation, force: bool):
    savefile = f'../data/{mol_name}_{localisation}.json'

    with open(savefile, 'w') as f:
        json.dump(result, f, indent=4)
        print(f'{savefile} created.')

In [ ]:
# acetonitrile
# N-methylmethanamine
# acetaldehyde
# formamide
# ethanamine
# ethanol
# methoxymethane
# fluoroethane
# acetone
# Isopropyl alcohol 

find_and_save('Isopropyl alcohol')

In [ ]:
import os
# get xyz file
notebook_dir = os.getcwd()
mol_structures_dir = os.path.join(notebook_dir, 'molecular_structures')

In [ ]:
from nbed.mol_plot import Draw_molecule

file =  os.path.join(mol_structures_dir, 'Isopropyl alcohol.xyz') 

with open(file, 'r') as f:
    xyz = f.read()

print(xyz)

viewer = Draw_molecule(xyz, width=400, height=400, style='sphere')
viewer.show()

In [ ]:
from pathlib import Path
from pyscf.gto import Mole
import nbed
from nbed.driver import NbedDriver
from nbed.ham_builder import HamiltonianBuilder
from nbed.ham_converter import HamiltonianConverter
from openfermion import count_qubits
import json
from pyscf import cc

localisation = ['spade','mullikan','boys','ipo']

# Need to build paths to xyz files.
max_atoms = 3

def get_results(mol_name, active_atoms, localisation='pipek-mezey', qubits=None, force=True):
    savefile = f'../data/{mol_name}_{localisation}.json'
    if Path(savefile).exists() and not force:
        in_string = input(f"Do you want to overwrite the saved data for {mol_name} [y]/n")
        if in_string != "y":
            print("Not saving.")
            return
    
    print(f"\nGetting results for:\n\t{mol_name}\n\t{active_atoms} atoms\n\t{localisation} localisation\n")
    find_and_save(mol_name)
    geometry = f'./molecular_structures/{mol_name}.xyz'

    config = {}
    # We pass these to the driver.
    config['localization']=localisation
    config['basis']='sto-3g'
    config['xc_functional']='b3lyp'
    config['convergence']=1e-6
    config['savefile']=None
    config['charge']=0
    config['mu_level_shift']=1e6
    config['run_ccsd_emb']=False
    config['run_fci_emb']=False
    config['max_ram_memory']=8_000
    config['pyscf_print_level']=1
    
    result = {}
    result.update(config)
    transform='jordan_wigner'

    for active in range(1,1+active_atoms):
        print(f"Starting driver with {active} active atoms.")
        driver = NbedDriver(geometry=geometry, n_active_atoms=active, projector='both', **config)
        print("Driver finished.")
        if active == 1:
            result['full'] = {}
            result['e_dft'] = driver._global_rks.e_tot
            full_builder = HamiltonianBuilder(driver._global_hf, constant_e_shift=driver._global_hf.energy_nuc(), transform=transform)
            full_ham = full_builder.build(qubits, taper=False)
            result['full']['qham'] = HamiltonianConverter(full_ham)._intermediate
            result['full']['terms'] = len(full_ham.terms)
            result['full']['n_qubits'] = count_qubits(full_ham)
            result['full']['classical_energy'] = 0
            result['full']['ground'] = None
            print('Full system Hamiltonian built.')


        result[active] = {}
        # Mu shift result
        mu_builder = HamiltonianBuilder(driver._mu['scf'], driver._mu['classical_energy'], transform=transform)
        qham = mu_builder.build(qubits, taper=False)
        result[active]['mu'] = {}
        result[active]['mu']['qham'] = HamiltonianConverter(qham)._intermediate
        result[active]['mu']['terms'] = len(qham.terms)
        result[active]['mu']['n_qubits'] = count_qubits(qham)
        result[active]['mu']['classical_energy'] = driver._mu['classical_energy']
        result[active]['mu']['ground'] = None
        print("Mu finished.")

        # Huzinaga result
        huz_builder = HamiltonianBuilder(driver._huzinaga['scf'], driver._huzinaga['classical_energy'], transform=transform)
        qham = huz_builder.build(qubits, taper=False)
        result[active]['huz'] = {}
        result[active]['huz']['qham'] = HamiltonianConverter(qham)._intermediate
        result[active]['huz']['terms'] = len(qham.terms)
        result[active]['huz']['n_qubits'] = count_qubits(qham)
        result[active]['huz']['classical_energy'] = driver._huzinaga['classical_energy']
        result[active]['huz']['ground'] = None
        print("Huzinaga finished.")


# untapered_mu = mu_builder.build(taper=False)
# untapered_huz = huz_builder.build(taper=False)
# untapered_full = full_builder.build(taper=False)
# print(f"Ground diff with tapering {result['mu']['ground']-ground(untapered_mu)}/{result['huz']['ground']-ground(untapered_huz)}")
# print(f"Untapered Qubits {count_qubits(untapered_mu)}/{count_qubits(untapered_huz)}/{count_qubits(untapered_full)}")
# print(f"Untapered Terms {len(untapered_mu.terms)}/{len(untapered_huz.terms)}/{len(untapered_full.terms)}")
        
        print_results(result, active)
            
    result['mol_name']=mol_name
    result['transform']=transform
    result['qubit_limit'] = qubits

    save_data(result, mol_name, localisation, force)
    print("\n")
    return result


In [ ]:
get_results("cyclopentane", 5, force=False, localisation='spade')

In [ ]:
# acetonitrile
# N-methylmethanamine
# acetaldehyde
# formamide
# ethanamine
# ethanol
# methoxymethane
# fluoroethane
# acetone
# isopropyl_alcohol 

mols = []
for mol in mols:
    get_results(mol, [1], force=False, localisation='ibo')

mols = ['N-methylmethanamine', 'acetonitrile', 'ethanol', 'fluoroethane', 'isopropyl_alcohol']
for mol in mols:
    get_results(mol, [2], force=False, localisation='ibo')

for mol in ['ethanamine', 'formamide', 'acetaldehyde']:
    get_results(mol, [3], force=False, localisation='ibo')

In [ ]:
with open('../data/cyclopentane_spade.json', 'r') as f:
    cyclopropane = json.load(f)

In [ ]:
whole_dft = cyclopropane['e_dft']
active_atoms = range(1,6)
cp_qubits = [cyclopropane[str(i)]['mu']['n_qubits'] for i in active_atoms]
cp_terms = [cyclopropane[str(i)]['mu']['terms'] for i in active_atoms]

In [ ]:
full_terms = cyclopropane['full']['terms']
full_n_qubits = cyclopropane['full']['n_qubits']

In [ ]:
fig, ax = plt.subplots()
# ax.hlines(whole_ccsd, min(active_atoms), max(active_atoms), colors='blue', linestyle='dashed', label='CCSD')
ax.hlines(whole_dft, min(active_atoms), max(active_atoms), colors='red', linestyle='dashed', label='DFT')
ax.plot(active_atoms, energies, label='Huzinaga CCSD Embedding')
ax.set_xlabel('Active Carbon Atoms')
ax.set_ylabel('Energy (Ha)')
ax.set_xticks(active_atoms)
ax.set_title('Embeddings of Cyclopropane')
fig.legend()
fig.show()

In [ ]:
fig, ax = plt.subplots()
ax2 = ax.twinx()
ax2.set_ylabel('Number of Qubits Used')
ax2.hlines(full_n_qubits, min(active_atoms), max(active_atoms), colors='red', linestyle='dashed', label=f'Full system: {full_n_qubits}')
ax2.plot(active_atoms, cp_qubits, linestyle='solid', color='red', label='n qubits')
ax.hlines(full_terms, min(active_atoms), max(active_atoms), colors='blue', linestyle='dotted', label=f'Full system: {full_terms}')
ax.plot(active_atoms, cp_terms, linestyle='solid', color='blue', label='terms')
ax.set_xlabel('Active Carbon Atoms')
ax.set_xticks(active_atoms)
ax.set_ylabel('Number of Terms')
ax.set_title('Embeddings of Cyclopropane')
fig.legend()
fig.show()

In [ ]:
# Extract data from the results json files

from pathlib import Path
import json

data = {}
for file in Path("../results/").iterdir():
    if "spade" in file.name:
        continue
    
    with open(file, 'r') as f:
        file_data = json.load(f)

    mol_name = file_data['mol_name']
    data[mol_name] = file_data
    data[mol_name]['full'].pop('qham')

    for i in range(5):
        n_data = data[mol_name].get(str(i), False)
        if n_data:
            n_data['huz'].pop('qham')
            n_data['mu'].pop('qham')

# Filters out anything too big to get the embedded ground state
data = {key: data[key] for key in sorted(data) if data[key]['1']['mu']['ground']}

In [ ]:
import pandas as pd
df = pd.DataFrame(data).T

In [ ]:
full_vals = pd.DataFrame(df['full'].to_list(), index=df['mol_name'])
one_mu = pd.DataFrame([v['mu'] for v in df['1'].to_list()], index=df['mol_name'])
one_huz = pd.DataFrame([v['huz'] for v in df['1'].to_list()], index=df['mol_name'])

In [ ]:
with open('../data/cyclopropane_spade.json', 'r') as f:
    propane = json.load(f)

In [ ]:
import numpy as np
mol_names = df['mol_name'].to_list()
full_vals = df['full'].to_list()
ccsd_vals = df['e_ccsdt'].to_list()

full_qubits = np.array([v['n_qubits'] for v in full_vals])
full_terms = np.array([v['terms'] for v in full_vals])

act_1 = df['1'].to_list()
mu_vals_1 = [v['mu'] for v in act_1]
huz_vals_1 = [v['huz'] for v in act_1]
mu_qubits_1 = np.array([v['mu']['n_qubits'] for v in act_1])
mu_terms_1 = [v['mu']['terms'] for v in act_1 if type(v) is dict]
huz_qubits_1 = np.array([v['huz']['n_qubits'] for v in act_1])
huz_terms_1 = [v['huz']['terms'] for v in act_1 if type(v) is dict]

act_2 = df['2'].to_list()
mu_vals_2 = [v['mu'] for v in act_2]
huz_vals_2 = [v['huz'] for v in act_2]
mu_qubits_2 = np.array([v['mu']['n_qubits'] for v in act_2])
mu_terms_2 = [v['mu']['terms'] for v in act_2 if type(v) is dict]
huz_qubits_2 = np.array([v['huz']['n_qubits'] for v in act_2])
huz_terms_2 = [v['huz']['terms'] for v in act_2 if type(v) is dict]

In [ ]:
energies_full = [v['ground'] if v['ground'] is not None else 0 for v in full_vals]
energies_dft = [v['e_dft'] if v['e_dft'] is not None else 0 for v in data.values()]
energies_mu = [v['ground'] if v['ground'] is not None else 0 for v in mu_vals_1]
energies_huz = [v['ground'] if v['ground'] is not None else 0 for v in huz_vals_1]

energies_full = np.array(energies_full)
energies_dft = np.array(energies_dft)
energies_mu = np.array(energies_mu)
energies_huz = np.array(energies_huz)
energies_ccsd = np.array(ccsd_vals)

dft_diffs = [100*(dft -ccsd)/ccsd if (dft and ccsd) else 0 for dft, ccsd in zip(energies_dft, energies_ccsd)]
mu_diffs = [100*(v -ccsd)/ccsd if (v and ccsd)  else 0 for v, ccsd in zip(energies_mu, energies_ccsd)]
huz_diffs = [100*(v -ccsd)/ccsd if (v and ccsd) else 0 for v, ccsd in zip(energies_huz, energies_ccsd)]



In [ ]:
import matplotlib.pyplot as plt
import numpy as np

x = np.arange(len(mol_names))  # the label locations
width = 0.1  # the width of the bars

fig, (ax1, ax2, ax3) = plt.subplots(3, figsize = (12,9))
def_es = ax1.bar(x + width, dft_diffs, width, label='DFT')
mu_es = ax1.bar(x, mu_diffs, width, label='$\mu$-shift')
huz_es = ax1.bar(x - width, huz_diffs, width, label='Huzinaga')

full_qubs = ax2.bar(x + width, full_qubits, width, label='Full system')
mu_qubs = ax2.bar(x, mu_qubits_1, width, label='$\mu$-shift')
huz_qubs = ax2.bar(x - width, huz_qubits_1, width, label='Huzinaga')

full_qubs = ax3.bar(x + width, full_terms, width, label='Full system')
mu_qubs = ax3.bar(x, mu_terms_1, width, label='$\mu$-shift')
huz_qubs = ax3.bar(x - width, huz_terms_1, width, label='Huzinaga')


# Add some text for labels, title and custom x-axis tick labels, etc.
ax1.set_ylabel("% Error from CCSD(T)")
ax1.set_title('Molecular ground state energies.')
ax1.set_xticks(x)
ax1.set_xticklabels(mol_names)
#ax1.tick_params(axis="x", rotation=50)
ax1.legend()

# Add some text for labels, title and custom x-axis tick labels, etc.
ax2.set_ylabel("Qubits")
ax2.set_title('Number of qubits in Hamiltonian.')
ax2.set_xticks(x)
ax2.set_xticklabels(mol_names)
ax2.tick_params(axis="x", rotation=50)
ax2.legend()

# Add some text for labels, title and custom x-axis tick labels, etc.
ax3.set_ylabel("Terms.")
ax3.set_title('Number of terms in Hamiltonian.')
ax3.set_xticks(x)
ax3.set_xticklabels(mol_names)
ax3.tick_params(axis="x", rotation=50)
ax3.legend()

def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')

for ax in fig.get_axes():
    ax.label_outer()

# autolabel(rects3)
# autolabel(rects2)

fig.tight_layout()
plt.show()

## General Params 
threshold
basis
xc functional


## Small molecules

One for SPADE
One for IBO
Molecule, active atoms(AOs), dft energy, embedding (huz/mu), fci/ccsd, mu: qubits(terms), huz: qubits(terms), full:qubits(terms)


Graphs:
Bar graph of molecules energies, hamiltonian size
Cyclic molecule: terms / qubits, energy
bonus: compare molecular orbitals


## H4
One for different numbers of active atoms 1-3
Angle, dft energy, embedding (huz/mu), full_system: fci/ccsd, mu: qubits(terms), huz: qubits(terms), full:qubits(terms)

## Water
Bond length, dft energy, embedding (huz/mu), fci/ccsd, mu: qubits(terms), huz: qubits(terms), full:qubits(terms)